In [7]:
%load_ext lab_black

In [8]:
from googleapiclient.discovery import build
import pandas as pd
import os

In [9]:
youTubeApiKey = _ # Read here
youtube = build("youtube", "v3", developerKey=youTubeApiKey)

# 1. Snippets

In [24]:
snippets = (
    youtube.search().list(part="snippet", type="channel", q="data science").execute()
)

In [25]:
len(snippets)

6

In [26]:
channelId = snippets["items"][0]["snippet"]["channelId"]

# 2. Stats

In [29]:
stats = youtube.channels().list(part="statistics", id=channelId).execute()

In [30]:
print(
    "Total Number of Subscribers of "
    + snippets["items"][0]["snippet"]["title"]
    + " is "
    + stats["items"][0]["statistics"]["subscriberCount"]
)
print(
    "Total Number of Videos Uploaded at "
    + snippets["items"][0]["snippet"]["title"]
    + " is "
    + stats["items"][0]["statistics"]["videoCount"]
)
print(
    "Total Number of view on all videos of "
    + snippets["items"][0]["snippet"]["title"]
    + " is "
    + stats["items"][0]["statistics"]["viewCount"]
)

Total Number of Subscribers of 365 Data Science is 221000
Total Number of Videos Uploaded at 365 Data Science is 198
Total Number of view on all videos of 365 Data Science is 8713351


# 3. Content

In [32]:
content = youtube.channels().list(id=channelId, part="contentDetails").execute()
content["items"]

[{'kind': 'youtube#channel',
  'etag': 'uufRzloPpY1P_2OHLGeCRJz7E24',
  'id': 'UCEBpSZhI1X8WaP-kY_2LLcg',
  'contentDetails': {'relatedPlaylists': {'likes': '',
    'uploads': 'UUEBpSZhI1X8WaP-kY_2LLcg'}}}]

In [37]:
UploadId = content["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]
print(
    "The Id of the upload playlist of this channel "
    + UploadId
    + " .In this playlist all the uploaded videos by this channels."
)

The Id of the upload playlist of this channel UUEBpSZhI1X8WaP-kY_2LLcg .In this playlist all the uploaded videos by this channels.


# 4. Status

In [33]:
status = youtube.channels().list(id=channelId, part="status").execute()
print(status)

{'kind': 'youtube#channelListResponse', 'etag': 'YDt6WRKagWRVNt6qxkpyIL6Kj_g', 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5}, 'items': [{'kind': 'youtube#channel', 'etag': 'GiWi_5TeeBeU21QVJJ08P102kTM', 'id': 'UCEBpSZhI1X8WaP-kY_2LLcg', 'status': {'privacyStatus': 'public', 'isLinked': True, 'longUploadsStatus': 'longUploadsUnspecified', 'madeForKids': False}}]}


# 5. Channel Selection

In [35]:
channelSections = (
    youtube.channelSections().list(channelId=channelId, part="snippet").execute()
)
channelSections["items"][0]

{'kind': 'youtube#channelSection',
 'etag': '9wbA-mNAt-xQarB2D4U0-VuAOOo',
 'id': 'UCEBpSZhI1X8WaP-kY_2LLcg.jNQXAC9IVRw',
 'snippet': {'type': 'recentuploads',
  'channelId': 'UCEBpSZhI1X8WaP-kY_2LLcg',
  'position': 0}}

# 6. Playlist Items

In [38]:
allVideos = []
nextPage_token = None
res = (
    youtube.playlistItems()
    .list(playlistId=UploadId, maxResults=50, part="snippet", pageToken=nextPage_token)
    .execute()
)
for _ in range(5):
    res = (
        youtube.playlistItems()
        .list(
            playlistId=UploadId, maxResults=50, part="snippet", pageToken=nextPage_token
        )
        .execute()
    )
    allVideos += res["items"]

    nextPage_token = res.get("nextPageToken")
    if nextPage_token is None:
        break

# print(allVideos)

In [39]:
print("Display the information related to the latest uploaded video on the channel\n")
print("Title of latest video uploaded: " + allVideos[0]["snippet"]["title"])
print("The latest video uploaded on date: " + allVideos[0]["snippet"]["publishedAt"])

Display the information related to the latest uploaded video on the channel

Title of latest video uploaded: Free Month at 365 Data Science
The latest video uploaded on date: 2021-10-15T14:09:38Z


# Video Stats

In [44]:
video_ids = list(map(lambda x: x["snippet"]["resourceId"]["videoId"], allVideos))
print(len(video_ids))

198


In [46]:
stats = []
for i in range(0, len(video_ids), 40):
    res = (
        (youtube)
        .videos()
        .list(id=",".join(video_ids[i : i + 40]), part="statistics")
        .execute()
    )
    stats += res["items"]

In [49]:
len(stats)

198

In [52]:
title = []
liked = []
# disliked = []
views = []
url = []
comment = []
videoid = []
publishedDate = []
video_description = []
for i, s in enumerate(stats):
    # print(i)
    title.append((allVideos[i])["snippet"]["title"])
    publishedDate.append((allVideos[i])["snippet"]["publishedAt"])
    video_description.append((allVideos[i])["snippet"]["description"])
    liked.append(int((stats[i])["statistics"]["likeCount"]))
    # disliked.append(int((stats[i])['statistics']['dislikeCount']))
    views.append(int((stats[i])["statistics"]["viewCount"]))
    comment.append(int((stats[i])["statistics"]["commentCount"]))
    videoid.append(allVideos[i]["snippet"]["resourceId"]["videoId"])

In [55]:
len(title), len(liked, publi)

(198, 198)

In [56]:
import pandas as pd

data = {
    "title": title,
    "videoIDS": videoid,
    "video_description": video_description,
    "publishedDate": publishedDate,
    "likes": liked,
    "views": views,
    "comment": comment,
}
df = pd.DataFrame(data)
df.head()

,title,videoIDS,video_description,publishedDate,likes,views,comment
0,Free Month at 365 Data Science,I2gT9lC3PXI,👉🏻 Download Our Free Data Science Career Guide...,2021-10-15T14:09:38Z,2074,19939,128
1,Who is a Data Scientist in 2021? A Research on...,0YDycxk1qgI,👉🏻 Download Our Free Data Science Career Guide...,2021-10-14T12:43:01Z,1333,50517,35
2,Say hello to 365 Data Science v2.0,6b4ABkIH9jQ,👉🏻 Download Our Free Data Science Career Guide...,2021-10-08T09:38:37Z,107,16865,19
3,Introduction to R Programming | 365 Data Scien...,3scFDnDuT14,👉🏻 Download Our Free Data Science Career Guide...,2021-09-08T13:08:28Z,295,7947,9
4,Introduction to Excel | 365 Data Science Courses,qM82K6IpVe4,👉🏻 Download Our Free Data Science Career Guide...,2021-09-02T07:58:06Z,180,10248,4


In [59]:
df["publishedDate"] = pd.to_datetime(df["publishedDate"])

In [61]:
df["channelId"] = channelId